In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential, layers
import numpy as np
from gym_derk.envs import DerkEnv
import nest_asyncio

nest_asyncio.apply()
from src.environments import EnvSix


In [ ]:
%load_ext autoreload
%autoreload 2

- Apprentissage génétique:
	1. Instancier 6 modèles Keras
	2. Associer chaque modèles à un agent
	3. Lancer un épisode avec les actions des modèles
	4. Recevoir les reward
	5. Garder les poids des modèles les plus performants
	6. Créer des variation du modèle le plus performant
	7. Recommencer étapes 2

In [110]:
class Agents:

    def __init__(self):
        self.models = [self.initialize_model() for _ in range(6)]

    def initialize_model(self):
        model = Sequential()
        model.add(layers.Dense(64, input_dim=64, activation='relu'))
        model.add(layers.Dense(30, activation='relu'))
        model.add(layers.Dense(20, activation='relu'))
        model.add(layers.Dense(15, activation='tanh'))
        return model

    def predict(self, obs):
        return np.array([self.models[i].predict(x=obs[i], verbose=0) for i in range(6)])

    def add_noise_to_weights(self, model):
        weights = model.get_weights()
        noise = []
        for weight in weights:
            noise.append(np.random.normal(0, 1, size=weight.shape))
        noisy_weights = [w + n for w, n in zip(weights, noise)]
        model.set_weights(noisy_weights)
        return model

    def train(self, rewards):
        self.save_model(self.models[np.argmax(rewards)], path=f"model{np.max(rewards)}.h5")
        self.mutate(np.argmax(rewards))
    
    def mutate(self, best_model_index):
        best_model = self.models[best_model_index]
        new_models = []
        # create 6 copies of the best model
        for _ in range(6):
            model = keras.models.clone_model(best_model)
            model = self.add_noise_to_weights(best_model)
            new_models.append(model)
        self.models = new_models
        
    def save_model(self, model, path):
        model.save(path)

In [111]:
env.close()

In [106]:
reward = {
    "damageEnemyStatue": 4,
    "damageEnemyUnit": 2,
    "killEnemyStatue": 4,
    "killEnemyUnit": 2,
    "healFriendlyStatue": 1,
    "healTeammatel": 2,
    "healTeammate2": 2,
    "timeSpentHomeBase": 0,
    "timeSpentHomeTerritory": 0,
    "timeSpentAwayTerritory": 0,
    "timeSpentAwayBase": 0,
    "damageTaken": -1,
    "friendlyFire": -1,
    "healEnemy": -1,
    "fallDamageTaken": -10,
    "statueDamageTaken": 0,
    "manualBonus": 0,
    "victory": 100,
    "loss": -100,
    "tie": 0,
    "teamSpirit": 0.5,
    "timeScaling": 1,
}

In [112]:
slot_1 = ['Blaster', 'ParalyzingDart', 'HeliumBubblegum']
slot_2 = ['Pistol', 'HealingGland', 'FrogLegs']

env = EnvSix(turbo_mode = True, reward_function=reward, home_team=[
    {
        'primaryColor': '#00ff00',
        'slots': ['Blaster', 'ParalyzingDart', 'HeliumBubblegum']
    },
    {
        'primaryColor': '#00ff00',
        'slots': ['Blaster', 'ParalyzingDart', 'HeliumBubblegum']
    },
    {
        'primaryColor': '#00ff00',
        'slots': ['Pistol', 'HealingGland', 'FrogLegs']
    },
],
              away_team=[
                  {
                      'primaryColor': '#00ff00',
                      'slots': ['Pistol', 'HealingGland', 'FrogLegs']
                  },
                  {
                      'primaryColor': '#00ff00',
                      'slots':
                      ['Blaster', 'ParalyzingDart', 'HeliumBubblegum']
                  },
                  {
                      'primaryColor': '#00ff00',
                      'slots': ['Pistol', 'HealingGland', 'FrogLegs']
                  },
              ])

In [ ]:
observations.shape

In [ ]:
observations = env.reset()
observations = observations.reshape(6,1,64)
actions = les_agents.predict(observations)
step_return = env.step(actions)

In [108]:
les_agents = Agents()

In [113]:
for _ in range(20):
    observations = env.reset()
    observations = observations.reshape(6,1,64)
    rewards = np.zeros(6)
    done = [False]
    while not all(done):
        actions = les_agents.predict(observations)
        observations, reward, done, info = env.step(actions)
        observations = observations.reshape(6,1,64)  
        rewards += reward
        if all(done):
            break
    print(np.mean(rewards))
    les_agents.train(rewards)
    

7.034779474139214
-36.66666793823242


In [ ]:
rewards

In [ ]:
step_return

In [ ]:
env.close()

In [ ]:
env.reset()